In [ ]:
sm = snakemake

In [ ]:
import pandas as pd

import spherpro.bro as spb
import spherpro.datastore as spd
import spherpro.library as spl
import spherpro.configuration as conf
import spherpro.db as db
import imp
import pycytools as pct
import pycytools.library
import re
import os

import numpy as np
import spherpro.library as lib
import matplotlib.pyplot as plt
import plotnine as gg
import seaborn as sns
from matplotlib import colors
#import mpld3
import pathlib
#import feather

In [ ]:
from src.variables import Vars
from src.config import Conf

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import spherpro.bromodules.helpers_anndata as ha

import rpy2
import os
#os.environ["R_HOME"] = "/home/vitoz/miniconda3/envs/spherpro2/lib/R"
%load_ext rpy2.ipython
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

### Aim: Load lmm results

In [ ]:
def censor_dat(x, q=99.99):
    x = np.copy(x)
    p = np.percentile(x,q=q)
    x[x> p] = p
    return x

def cur_logtransf(x):
    return np.log10(x+0.1)
transform = cur_logtransf

def cur_transf(x):
    x= censor_dat(x)
    x= cur_logtransf(x)
    return x

In [ ]:
class CurVariableHelper(Vars):
    pass
    
V = CurVariableHelper

In [ ]:
class C(Conf):
    fn_config = sm.input.fn_config
    fol_plts = pathlib.Path(sm.output.fol_figures)
    fol_paper = fol_plts 
    fn_out_sig = sm.output.fn_out_sig
    fns_lmm_params = sm.input.fns_lmm_params
    fns_lmm_model = sm.input.fns_lmm_model
    fns_lmm_absstat = sm.input.fns_lmm_absstat
    fns_lmm_randeff = sm.input.fns_lmm_randeff
    fn_panel_ordered = sm.input.fn_panel_ordered
    fn_constructmeta = sm.input.fn_construct_meta
    fn_phist = fol_plts / 'phist.png'

In [ ]:
C.fol_plts.mkdir(exist_ok=True)
C.fol_paper.mkdir(exist_ok=True)

In [ ]:
C.fns_lmm_params

In [ ]:
bro = spb.get_bro(C.fn_config)

In [ ]:
import spherpro.bromodules.plot_condition_images as pci
ci = pci.PlotConditionImages(bro)

Load the fitted data

In [ ]:
bro.data._read_pannel()
dat_pannel = bro.data.pannel

fil = bro.filters.measurements.get_measmeta_filter_statements(channel_names=[tuple(dat_pannel.query(f'{V.COL_WORKING} == True')['metal'].values), 'object'],
                                                         measurement_names=['MeanIntensityComp', 'dist-rim'],
                                                             measurement_types=['Intensity', 'Location'],
                                                             stack_names=['FullStackFiltered', 'ObjectStack'])
q_meas = (bro.data.get_measmeta_query()
         .add_columns(db.ref_planes.channel_name,
                      db.stacks.stack_name,
                     db.ref_stacks.scale)
          .filter(fil)
         )
dat_measmeta = bro.doquery(q_meas).merge(dat_pannel,left_on='channel_name', right_on='metal')

In [ ]:
chans = [c for c in dat_measmeta[V.COL_CHANNELNAME].unique() if c != 'object']

In [ ]:
dat_lmcoefs = (pd.concat([pd.read_csv(sm.params.pat_lmm_params.format(channel=c)) for c in chans],
                        keys=chans, names=[V.COL_CHANNELNAME,'index']).reset_index(V.COL_CHANNELNAME, drop=False)
               .reset_index(drop=True)
                )

In [ ]:
dat_absstat= (pd.concat([pd.read_csv(sm.params.pat_lmm_absstat.format(channel=c)) for c in chans],
                        keys=chans, names=[V.COL_CHANNELNAME,'index']).reset_index(V.COL_CHANNELNAME, drop=False)
               .reset_index(drop=True)
                )

In [ ]:
from pandas.api.types import CategoricalDtype
dat_lmcoefs[V.COL_ISNB] = dat_lmcoefs[C.FIL_LM].astype(CategoricalDtype([1,2,3])).cat.rename_categories(C.FIL_LM_CLASSES[1:])
dat_absstat[V.COL_ISNB] = dat_absstat[C.FIL_LM].astype(CategoricalDtype([1,2,3])).cat.rename_categories(C.FIL_LM_CLASSES[1:])

In [ ]:
dat_lmcoefs = dat_lmcoefs.rename(columns={'Pr(>|t|)': V.COL_P,
                                         '2.5 %': V.COL_LB,
                                         '97.5 %': V.COL_UB,
                                          V.COL_ESTIMATE: V.COL_DELTA
                                         })

In [ ]:
dat_lmcoefs[V.COL_FC] = 10**dat_lmcoefs[V.COL_DELTA]
fil = (~dat_lmcoefs[V.COL_CONDNAME].isna())



In [ ]:
dat_lmcoefs

In [ ]:
import statsmodels.stats.multitest as multitest

In [ ]:
fil = (~dat_lmcoefs[V.COL_CONDNAME].isna())
dat_lmcoefs.loc[fil, V.COL_P_CORR] = multitest.multipletests(dat_lmcoefs.loc[fil, V.COL_P].values, alpha=0.1, method='FDR_BH')[1]

In [ ]:
fig = plt.figure()
dat_lmcoefs.loc[fil, V.COL_P].hist(bins=50)
plt.title('p-value histogram')
plt.xlabel('p-value')
plt.ylabel('occurence in bin')

In [ ]:
fig.savefig(C.fn_phist )

In [ ]:
p_tresh = 0.01
deltat = np.log10(1.3)
fil = (dat_lmcoefs[V.COL_P_CORR] < p_tresh)& (np.abs(dat_lmcoefs[V.COL_DELTA]) > deltat) & (~dat_lmcoefs[V.COL_CONDNAME].isna())

sighits = dat_lmcoefs.loc[fil,:].merge(dat_measmeta).sort_values(V.COL_DELTA)

In [ ]:
#sighits.loc[(sighits['goodname'].isin(['GFP','DYKDDDDK Tag'])==False),:]

In [ ]:
sighits

In [ ]:
sighits.loc[sighits[V.COL_CONDNAME].isin(['GFP_GFP-FLAG','GFP_nan'])].merge(dat_measmeta)

In [ ]:
dat_plt = dat_lmcoefs.copy()

In [ ]:
co_fc = 1.05
co_p = 2

maxfc = 4
breaks = 10**np.arange(np.log10(1/maxfc), np.log10(maxfc)+0.2, 0.2)

pltdat = dat_plt.dropna().merge(dat_measmeta[[V.COL_CHANNELNAME, V.COL_MEASNAME, V.COL_GOODNAME]]).copy()
#c = ['GFP_GFP-FLAG', 'GFP_nan']
#pltdat = pltdat.query(f'{V.COL_CONDNAME} in {c}')
pltdat['sig'] = ((np.abs(pltdat['delta']) > np.log10(co_fc)) & (pltdat[V.COL_P_CORR] < 10 ** (-co_p)))
pltdat['sigdir'] = pd.Categorical((pltdat['sig'] * np.sign(pltdat['delta'])).astype(str), categories=['-1','1'])
pltdat.loc[pltdat[V.COL_FC] >maxfc, V.COL_FC] = maxfc
pltdat.loc[pltdat[V.COL_FC] <1/maxfc, V.COL_FC] = 1/maxfc
pltdatsig =pltdat.query('sig != 0')
pltdatnonsig =pltdat.query('sig == 0')

p = (gg.ggplot(pltdatnonsig,
           gg.aes(x=V.COL_FC, y='-np.log10('+V.COL_P_CORR+'+10**(-10))'))+
         gg.facet_grid(f'{V.COL_ISNB}~{V.COL_CONDNAME}')+
     gg.geom_point()+
        gg.geom_vline(xintercept=co_fc, color='grey')+
      gg.geom_vline(xintercept=1/co_fc, color='grey')+
     gg.geom_hline(yintercept=co_p, color='grey')+
     gg.geom_point(gg.aes(color='sigdir'), data= pltdatsig)+

     gg.geom_text(gg.aes(label='goodname', color='sigdir'), size=8, data= pltdatsig)+
    gg.scale_x_log10(limits=(1/maxfc, maxfc), breaks=breaks)+
        gg.theme(figure_size=(60,6))
)

%%R -i pltdat -w 18200 -h 800 -u px


co_fc = 1.2
co_p = -log10(0.01)
require(ggplot2)
require(ggrepel)
fil = (abs(pltdat$delta) > log10(co_fc)) & (pltdat$p_corrected < 10^(-co_p))

ggplot(pltdat, aes(x=fc, y=(-log10(p_corrected+10^(-10)))))+
    facet_grid(isnb~condition_name)+
    geom_vline(aes(xintercept=co_fc), alpha=0.3)+
    geom_vline(aes(xintercept=(1/co_fc)), alpha=0.3)+
    geom_hline(aes(yintercept=co_p), alpha=0.3)+
    scale_x_continuous(trans='log2', breaks= c(1/8, 1/4,1/2,1, 2,4,8))+
    scale_color_hue(l=10)+
    geom_point()+
    geom_text_repel(data=pltdat[fil,],aes(label = goodname, color=fc>1), size=5)+
    xlab('log2(Fold Change)')+
    ylab('-log10(p)')+
    theme_grey(20)

In [ ]:
def get_pltname(d):
    return ' - '.join([d[V.COL_GOODNAME]])
                      
pltdat[V.COL_PLTNAME] = pltdat.apply(get_pltname , axis=1)

In [ ]:
pltdat[V.COL_ISNB] = pd.Categorical(pltdat[V.COL_ISNB], categories=['oexp', 'oexp-NB', 'ctrl'])

valvar = V.COL_DELTA
ind = [V.COL_CONDNAME, V.COL_ISNB]
col = V.COL_PLTNAME
dat_hm = (pltdat
          #.query(f'{V.COL_N_OVEREXPR} > 10')
          .pivot_table(valvar, index=ind, columns=col)
         )

In [ ]:
pltdat.query('isnb == "ctrl" & channel_name == "La139"')

Calculate clustering on the intrenal markers

In [ ]:
import scipy.cluster.hierarchy as hclust
import scipy.spatial.distance as dist
import scipy.stats as stats
from matplotlib.patches import Rectangle

import matplotlib.colors as mplcolors
import matplotlib as mpl
from matplotlib.colors import LogNorm


def adapt_linkage(clink, grp_values_old, grp_values_new):
    """
    Adapts a linkage matrix for a matrix where there can be multiple values per group
    """
    # total number of groups need to be compatible
    assert(len(grp_values_old) == clink[-1,3])
    # start making the 'new' cluster table, by assigning all the observations that belong to the same group
    # together
    clink = clink.copy()
    new_lmat = list()
    n_newleafes = len(grp_values_new)
    n_oldleafes = len(grp_values_old)
    c_dict = np.zeros(n_oldleafes)
    for idx, val in enumerate(grp_values_old):
        new_vals = np.where(grp_values_new == val)[0][::-1].tolist()
        new_validx = new_vals.pop()
        while (len(new_vals) >= 1):
            n_row = np.array([[new_validx, new_vals.pop(), 0, 0]])
            new_lmat.append(n_row)
            new_validx = len(new_lmat)+n_newleafes-1
        
        c_dict[idx] = new_validx
    # adapt the old linkage matrix
    # update the columns
    for i in [0,1]:
        fil = clink[:, i] > (n_oldleafes-1)
        clink[fil, i] += len(new_lmat)+n_newleafes-n_oldleafes
        fil = fil == False
        clink[fil, i] = c_dict[clink[fil, i].astype(np.int)]
    new_clink = np.concatenate([np.concatenate(new_lmat, axis=0), clink])
    leaves_link = np.zeros((n_newleafes,4))
    
    links_all = np.concatenate([leaves_link,new_clink])
    links_all[:,3] = 1
    
    # fix the leaf counts:
    for i in range(links_all.shape[0]):
        if i > n_newleafes-1:
            lidx= int(links_all[i,0])
            ridx = int(links_all[i,1])
            links_all[i,3] = links_all[lidx,3] + links_all[ridx,3]         
    new_clink = links_all[n_newleafes:,:]
    return new_clink


class MidPointLogNorm(LogNorm):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        LogNorm.__init__(self,vmin=vmin, vmax=vmax, clip=clip)
        self.midpoint=midpoint
    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [np.log(self.vmin), np.log(self.midpoint), np.log(self.vmax)], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(np.log(value), x, y))

Calculate the row clustering per condition

-> remove GFP & FLAG tag to estimate the clustering

In [ ]:
c_method = 'ward'
c_link = 'euclidean'

In [ ]:
mlist = ['Er167', 'Tm169', 'Yb176']

fil_cs = [c for c in dat_hm.columns.get_level_values(V.COL_PLTNAME) for m in mlist if m in c]
fil = dat_hm.columns.get_level_values(V.COL_PLTNAME).isin(fil_cs)

#cmat = dat_hm.xs('oexp',  level=V.COL_ISNB)
cmat = dat_hm.loc[:, fil==False].unstack(level=V.COL_ISNB).fillna(0)
rlink =  hclust.linkage(cmat, method=c_method, metric=c_link, optimal_ordering=True)
rnames = cmat.index.get_level_values(V.COL_CONDNAME)

Calculate the column clustering by channel


In [ ]:
dat_hm_plot = (dat_hm.loc[dat_hm.index.get_level_values(V.COL_CONDNAME).isin(rnames),:]
               .unstack(level=V.COL_ISNB)
               .fillna(0)
               .stack(level=V.COL_ISNB)
              )

In [ ]:
cmat_c = dat_hm_plot #.xs('oexp',  level=V.COL_ISNB)
clink =  hclust.linkage(cmat_c.T, method=c_method, metric=c_link, optimal_ordering=True)
cnames = cmat_c.columns.get_level_values(V.COL_PLTNAME)

row_linkage = adapt_linkage(rlink, rnames, dat_hm_plot.index.get_level_values(V.COL_CONDNAME))
col_linkage = clink

In [ ]:
def do_heatmap(pdat, col_linkage, row_linkage, figsize=(14,25),row_colors=True,**kwargs):
    tresh = 3.

    #pdat = dat_hm_plot.copy()
    pdat = np.log2(10**pdat)

    nticks = 5
    cbar_ticks = [np.round((np.log2(1/tresh)+i*(np.log2(tresh)-np.log2(1/tresh))/(nticks-1)), decimals=2) for i in range(nticks)]
    if row_colors:
        vals = pdat.index.get_level_values(V.COL_ISNB).codes
        cols = [mplcolors.get_named_colors_mapping()[c] for c in ['green','blue','darkgrey']]
        row_colors = [cols[i] for i in  vals]
    else:
        row_colors=None
        
    cg = sns.clustermap(pdat.reset_index(V.COL_ISNB, drop=True),
                        cmap="RdBu_r",
                        figsize= figsize,
                        col_linkage=col_linkage,
                        row_linkage=row_linkage,
                        center=0,
                        yticklabels=3,
                        xticklabels=1,
                        row_colors=row_colors,
                        #norm=LogNorm(vmin=1/tresh, vmax=tresh),
                        vmin=np.log2(1/tresh),
                        vmax=np.log2(tresh),
                        cbar_kws={'ticks': cbar_ticks},**kwargs

                       )
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0, va='top')
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), rotation=-90,rotation_mode='anchor', ha='left', va='center')

    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), size=5)
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), size=5)
    cg.fig.subplots_adjust(bottom=0.3)
    cg.fig.subplots_adjust(right=0.7)
        #sns.set(font_scale=0.2)
    cg.ax_heatmap.hlines(np.arange(3,dat_hm.shape[0],3), *cg.ax_heatmap.get_xlim(), lw=0.5,colors='grey')
    cg.ax_heatmap.vlines(np.arange(1,dat_hm.shape[1],1), *cg.ax_heatmap.get_ylim(), lw=0.5, colors='grey')
    return cg
cg = do_heatmap(dat_hm_plot.loc[:, cnames],col_linkage, row_linkage)

In [ ]:
delta_co = np.log10(1.1)
pltdat[V.COL_ISSIG] = (pltdat[V.COL_P_CORR] <= 0.01) & (np.abs(pltdat[V.COL_DELTA]) > delta_co) & (pltdat[V.COL_P] <= 0.01)

dat_hm_sig = pltdat.pivot_table(values=V.COL_ISSIG, index=dat_hm_plot.index.names, columns=dat_hm_plot.columns.names).fillna(False)

row_ind = dat_hm_plot.index[cg.dendrogram_row.reordered_ind]
col_ind = dat_hm_plot.columns[cg.dendrogram_col.reordered_ind]
dat_hm_sig_reord = dat_hm_sig.reindex(index=row_ind,columns=col_ind, fill_value=False)
coord = np.where(dat_hm_sig_reord)
coord = zip(coord[1], coord[0])

ax = cg.ax_heatmap

for (x,y) in coord:
    isnb = dat_hm_sig_reord.index.get_level_values(V.COL_ISNB)[y]
    #if isnb == 'ctrl':
    #    color = 'black'
    #elif isnb == 'oexp':
    #    color = 'red'
    #else:
    #    color='green'
    color='yellow'
    ax.scatter(x+0.5, y+0.4, color=color,zorder=2, s=14,edgecolors='black',linewidth=0.5)
cg.fig

Lodat the 'absolute increase' data

In [ ]:
V.COL_IS_CHANGED = 'is_changed'

In [ ]:
V.COL_FC_VS_OEXP = 'fc_vs_oexp'
V.COL_MEAN_ABS_DELTA = 'mean_abs_delta'

In [ ]:
d=(dat_absstat.query(f'{V.COL_CHANNELNAME} in ["Er167", "Tm169"]')
   .set_index(V.COL_ISNB)
   .sort_values(V.COL_FC_VS_OEXP, ascending=False)
     ['fc_vs_oexp'])
d.groupby(V.COL_ISNB).describe()

In [ ]:
dat_absstat[V.COL_IS_CHANGED] = dat_absstat.apply(lambda x: (x[V.COL_MEAN_ABS_DELTA] < 0) |
                                                  (x[V.COL_ISNB] == 'oexp') |
                                                   ((x[V.COL_ISNB] == 'oexp-NB') & (x[V.COL_FC_VS_OEXP] > (0.1))) |
                                                  ((x[V.COL_ISNB] == 'ctrl') & (x[V.COL_FC_VS_OEXP] > (0.05))) |
                                                    (np.isnan(x[V.COL_FC_VS_OEXP])) ,axis=1)

In [ ]:
dat_absstat.query('channel_name == "La139" & isnb=="ctrl"')

In [ ]:
def add_annotations(cg, pltdat, dathm, s=14):
    dat_hm_sig = pltdat.pivot_table(values=V.COL_ISSIG, index=dat_hm_plot.index.names, columns=dat_hm_plot.columns.names).fillna(False)

    if cg.dendrogram_row is not None:
        row_ind = dathm.index[cg.dendrogram_row.reordered_ind]
    else:
        row_ind = dathm.index
    if cg.dendrogram_col is not None:
        col_ind = dathm.columns[cg.dendrogram_col.reordered_ind]
    else:
        col_ind = dathm.columns
    dat_hm_sig_reord = dat_hm_sig.reindex(index=row_ind,columns=col_ind, fill_value=False)
    coord = np.where(dat_hm_sig_reord)
    coord = zip(coord[1], coord[0])

    ax = cg.ax_heatmap

    for (x,y) in coord:
        isnb = dat_hm_sig_reord.index.get_level_values(V.COL_ISNB)[y]
        #if isnb == 'ctrl':
        #    color = 'black'
        #elif isnb == 'oexp':
        #    color = 'red'
        #else:
        #    color='green'
        color='yellow'
        ax.scatter(x+0.5, y+0.4, color=color,zorder=2, s=s,edgecolors='black',linewidth=0.5)
    return cg



Do final cutoffs

In [ ]:
delta_co = np.log10(1.2)
dat_issig = pltdat.merge(dat_absstat[[V.COL_CHANNELNAME, V.COL_CONDNAME, V.COL_ISNB, V.COL_IS_CHANGED]],
                        on=[V.COL_CHANNELNAME, V.COL_CONDNAME, V.COL_ISNB])
dat_issig[V.COL_ISSIG] = ((dat_issig[V.COL_P_CORR] <= 0.01) & (np.abs(dat_issig[V.COL_DELTA]) > delta_co) &\
    (dat_issig[V.COL_P] <= 0.01) & (dat_issig[V.COL_IS_CHANGED] == True))


cg =do_heatmap(dat_hm_plot.loc[:, cnames],col_linkage, row_linkage)

cg = add_annotations(cg, dat_issig,dat_hm_plot.loc[:, cnames] ) 
cg.fig

In [ ]:
dat_issig['sigdir'] = dat_issig['sig'] * np.sign(dat_issig['delta'])

In [ ]:
class CHANS:
    p38 = 'Gd156'
    jnk = 'Er170'
    gfp1 = 'Er167'
    gfp2 = 'Tm169'
    flag = 'Yb176'
    stereotypic = [p38, jnk, gfp1, gfp2, flag]

In [ ]:
V.COL_ISSTEREOTYPIC = 'is_stereotypic'

In [ ]:
dstat = (dat_issig
     .assign(**{V.COL_ISSTEREOTYPIC: lambda x: ['gfp' if chan in [CHANS.gfp1, CHANS.gfp2, CHANS.flag] else
                                      'p38' if chan == CHANS.p38 else
                                      'JNK' if chan == CHANS.jnk else
                                      'other' for chan in  x[V.COL_CHANNELNAME].values]})
     .pivot_table(index=V.COL_CONDNAME, columns=[V.COL_ISSTEREOTYPIC, V.COL_ISNB], values=V.COL_ISSIG, aggfunc=np.sum)
     .sort_values([('other', 'oexp')], ascending=False)
)

dstat_all = (dat_issig
     .assign(**{V.COL_ISSTEREOTYPIC: lambda x: ['gfp' if chan in [CHANS.gfp1, CHANS.gfp2, CHANS.flag] else
                                      'other' for chan in  x[V.COL_CHANNELNAME].values]})
     .pivot_table(index=V.COL_CONDNAME, columns=[V.COL_ISSTEREOTYPIC, V.COL_ISNB], values=V.COL_ISSIG, aggfunc=np.sum)
     .sort_values([('other', 'oexp')], ascending=False)
)

In [ ]:
dstat.head()

Plot everything that has also neighbourhood effects

Order by pathway

In [ ]:
dat_pannel_ord = pd.read_csv(C.fn_panel_ordered)
COL_CLASS = 'class'
COL_ORD_WITHIN = 'ord_within_class'
cols = [V.COL_METAL, COL_CLASS, COL_ORD_WITHIN]
class_ord = ['tag','egf','mtor', 'cellcycle', 'stress', 'apoptosis','total']
dat_pannel_ord[COL_CLASS] = pd.Categorical(dat_pannel_ord[COL_CLASS], categories=class_ord)

ord_pltname = pltdat.drop_duplicates([V.COL_PLTNAME]).merge(dat_pannel_ord[[V.COL_METAL, COL_CLASS, COL_ORD_WITHIN]], left_on=V.COL_CHANNELNAME,
                             right_on=V.COL_METAL).sort_values([ COL_CLASS, COL_ORD_WITHIN])[V.COL_PLTNAME].values

In [ ]:
ord_pltname

In [ ]:
ord_pltname

In [ ]:
#col_nbresp = (dstat['other'].loc[(dstat[[(s, n) for s in ['other'] for n in ('oexp-NB','ctrl')]].sum(1) >1 ) |\
#                                 (dstat.index.get_level_values(V.COL_CONDNAME).isin([
#                                        'GFP_nan',
#                                        'GFP_GFP-FLAG',
#                                        'HcRed_GFP-FLAG',
#                                        'Luciferase_GFP-FLAG'])), :].index)

#col_nbresp = (dstat['other'].loc[((dstat['other']['oexp'] >0 ) |&\
#                                  (dstat['other']['oexp-NB'] >0 )) | \
#                                 dstat.index.get_level_values(V.COL_CONDNAME).isin([
#                                        'GFP_nan',
#                                        'GFP_GFP-FLAG',
#                                        'HcRed_GFP-FLAG',
#                                        'Luciferase_GFP-FLAG']), :].index)
col_nbresp = (dstat['other'].loc[(((dstat['other']['ctrl'] > 0 ) |\
                                  (dstat['other']['oexp-NB'] >0 )) | \
                                 dstat.index.get_level_values(V.COL_CONDNAME).isin([
                                        'GFP_nan',
                                        'GFP_GFP-FLAG',
                                        'HcRed_GFP-FLAG',
                                        'Luciferase_GFP-FLAG'])
                                 ) & ~(dstat['other']['oexp'].isna()), :].index)

mlist = ['Er167', 'Tm169', 'Yb176']


d= dat_hm_plot.loc[col_nbresp,:]
fil_cs = [c for c in d.columns.get_level_values(V.COL_PLTNAME) for m in mlist if m in c]
fil = d.columns.get_level_values(V.COL_PLTNAME).isin(fil_cs)
#cmat = dat_hm.xs('oexp',  level=V.COL_ISNB)
cmat = d.loc[:, fil==False].unstack(level=V.COL_ISNB).dropna()
rlink2 =  hclust.linkage(cmat, method=c_method, metric=c_link, optimal_ordering=True)
rnames2 = cmat.index.get_level_values(V.COL_CONDNAME)

row_linkage2 = adapt_linkage(rlink2, rnames2, d.index.get_level_values(V.COL_CONDNAME))

cmat_c = d
clink2 =  hclust.linkage(cmat_c.T, method=c_method, metric=c_link, optimal_ordering=True)


cnames2 = list(ord_pltname)

with plt.rc_context({'font.size': 5}):
    cg =do_heatmap(d.loc[:,cnames2], clink2, row_linkage2, figsize=(5, 5), col_cluster=False)


    cg = add_annotations(cg, dat_issig,d.loc[:,cnames2], s=5) 
    cg.fig.savefig(C.fol_paper / 'fig5_res_heatmap.pdf')
    cg.fig

In [ ]:
pdat = d.loc[:,cnames2]
pdat

In [ ]:
#col_nbresp = (dstat['other'].loc[((dstat['other']['oexp'] >0 )&\
#                                  (dstat['other']['oexp-NB'] ==0)) | \
#                                 dstat.index.get_level_values(V.COL_CONDNAME).isin([
#                                        'GFP_nan',
#                                        'GFP_GFP-FLAG',
#                                        'HcRed_GFP-FLAG',
#                                        'Luciferase_GFP-FLAG']), :].index)

mlist = ['Er167', 'Tm169', 'Yb176']


fil = ~dat_hm_plot.index.get_level_values(V.COL_CONDNAME).isin(col_nbresp)
d= dat_hm_plot.loc[fil,:][ord_pltname]
fil_cs = [c for c in d.columns.get_level_values(V.COL_PLTNAME) for m in mlist if m in c]
fil = d.columns.get_level_values(V.COL_PLTNAME).isin(fil_cs)
#cmat = dat_hm.xs('oexp',  level=V.COL_ISNB)
cmat = d.loc[:, fil==False].unstack(level=V.COL_ISNB).fillna(0)
rlink2 =  hclust.linkage(cmat, method=c_method, metric=c_link, optimal_ordering=True)
rnames2 = cmat.index.get_level_values(V.COL_CONDNAME)

cmat_c = d
clink =  hclust.linkage(cmat_c.T, method=c_method, metric=c_link, optimal_ordering=True)


cnames2 = ord_pltname

row_linkage2 = adapt_linkage(rlink2, rnames2, d.index.get_level_values(V.COL_CONDNAME))
col_linkage2 = clink

with plt.rc_context({'font.size': 5}):
    cg =do_heatmap(d.loc[:,cnames2],col_linkage2, row_linkage2, figsize=(5, 5),col_cluster=False)

    cg = add_annotations(cg, dat_issig,d.loc[:,cnames2], s=5) 
    cg.fig.savefig(C.fol_paper / 'fig5_res_heatmap_a.pdf')
    cg.fig

In [ ]:
col_nbresp = (dstat['other'].loc[((dstat['other']['oexp'] >1 )&\
                                  (dstat['other']['oexp-NB'] >1 )) | \
                                 dstat.index.get_level_values(V.COL_CONDNAME).isin([
                                        'GFP_nan',
                                        'GFP_GFP-FLAG',
                                        'HcRed_GFP-FLAG',
                                        'Luciferase_GFP-FLAG']), :].index)

mlist = ['Er167', 'Tm169', 'Yb176']



d= dat_hm_plot[ord_pltname]
fil_cs = [c for c in d.columns.get_level_values(V.COL_PLTNAME) for m in mlist if m in c]
fil = d.columns.get_level_values(V.COL_PLTNAME).isin(fil_cs)
#cmat = dat_hm.xs('oexp',  level=V.COL_ISNB)
cmat = d.loc[:, fil==False].unstack(level=V.COL_ISNB).dropna()
rlink2 =  hclust.linkage(cmat, method=c_method, metric=c_link, optimal_ordering=True)
rnames2 = cmat.index.get_level_values(V.COL_CONDNAME)

row_linkage2 = adapt_linkage(rlink2, rnames2, d.index.get_level_values(V.COL_CONDNAME))

cg =do_heatmap(d.loc[:,ord_pltname],None, row_linkage2, figsize=(12, 16), col_cluster=False)


cg = add_annotations(cg, dat_issig,d.loc[:,cnames2], s=10) 
cg.fig.savefig(C.fol_paper / 'supfig_oexpall_heatmap.pdf')
cg.fig

In [ ]:
col_nbresp = (dstat['other'].loc[((dstat['other']['oexp'] >0 )&\
                                  (dstat['other']['oexp-NB'] >0 )) | \
                                 dstat.index.get_level_values(V.COL_CONDNAME).isin([
                                        'GFP_nan',
                                        'GFP_GFP-FLAG',
                                        'HcRed_GFP-FLAG',
                                        'Luciferase_GFP-FLAG']), :].index)

mlist = ['Er167', 'Tm169', 'Yb176']



d= dat_hm_plot.loc[col_nbresp,:]
fil_cs = [c for c in d.columns.get_level_values(V.COL_PLTNAME) for m in mlist if m in c]
fil = d.columns.get_level_values(V.COL_PLTNAME).isin(fil_cs)
#cmat = dat_hm.xs('oexp',  level=V.COL_ISNB)
cmat = d.loc[:, fil==False].unstack(level=V.COL_ISNB).dropna()
rlink2 =  hclust.linkage(cmat, method=c_method, metric=c_link, optimal_ordering=True)
rnames2 = cmat.index.get_level_values(V.COL_CONDNAME)

cmat_c = d
clink =  hclust.linkage(cmat_c.T, method=c_method, metric=c_link, optimal_ordering=True)
cnames2 = cmat_c.columns.get_level_values(V.COL_PLTNAME)

row_linkage2 = adapt_linkage(rlink2, rnames2, d.index.get_level_values(V.COL_CONDNAME))
col_linkage2 = clink
cg =do_heatmap(d.loc[:,cnames2],col_linkage2, row_linkage2, figsize=(12, 10))
delta_co = np.log10(1.2)

cg = add_annotations(cg, dat_issig,d.loc[:,cnames2], s=10) 
cg.fig

In [ ]:
col_nbresp = (dstat['other'].loc[((dstat['other']['oexp'] >1 )&\
                                  (dstat['other']['oexp-NB'] >-1 )) | \
                                 dstat.index.get_level_values(V.COL_CONDNAME).isin([
                                        'GFP_nan',
                                        'GFP_GFP-FLAG',
                                        'HcRed_GFP-FLAG',
                                        'Luciferase_GFP-FLAG']), :].index)

mlist = ['Er167', 'Tm169', 'Yb176']



d= dat_hm_plot.loc[col_nbresp,:]
fil_cs = [c for c in d.columns.get_level_values(V.COL_PLTNAME) for m in mlist if m in c]
fil = d.columns.get_level_values(V.COL_PLTNAME).isin(fil_cs)
#cmat = dat_hm.xs('oexp',  level=V.COL_ISNB)
cmat = d.loc[:, fil==False].unstack(level=V.COL_ISNB).fillna(0)
rlink2 =  hclust.linkage(cmat, method=c_method, metric=c_link, optimal_ordering=True)
rnames2 = cmat.index.get_level_values(V.COL_CONDNAME)

cmat_c = d
clink =  hclust.linkage(cmat_c.T, method=c_method, metric=c_link, optimal_ordering=True)
cnames2 = cmat_c.columns.get_level_values(V.COL_PLTNAME)

row_linkage2 = adapt_linkage(rlink2, rnames2, d.index.get_level_values(V.COL_CONDNAME))
col_linkage2 = clink
cg =do_heatmap(d.loc[:,cnames2],col_linkage2, row_linkage2, figsize=(10, 15))
delta_co = np.log10(1.2)

cg = add_annotations(cg, dat_issig,d.loc[:,cnames2]) 
cg.fig

## Calculate statistics from these significant constructs

This is the main basis for the text

In [ ]:
dat_issig.to_csv(C.fn_out_sig, index=False)

In [ ]:
V.COL_CONSTR_TYPE = 'construct_type'

In [ ]:


dat_constrmeta = pd.read_csv(C.fn_constructmeta)

In [ ]:
dat_constrmeta

In [ ]:
dat_pannel = bro.data.pannel

In [ ]:
dat_constrmeta

In [ ]:
dat_pannel = bro.data.pannel

In [ ]:
d = (dat_issig
 .query(f'{V.COL_ISNB} == "oexp"')
 .groupby([V.COL_CHANNELNAME, V.COL_GOODNAME])[V.COL_ISSIG]
 .apply(lambda x: np.mean(x>0))
 .sort_values(ascending=False).reset_index()
)
d.query(f'{V.COL_CHANNELNAME} in {[CHANS.p38, CHANS.jnk]}')

-> p38 is changed in 84%, JNK in 64%

In [ ]:
(dat_issig
 .query(f'{V.COL_ISNB} == "oexp"')
 .query(f'{V.COL_CHANNELNAME} in {[CHANS.p38, CHANS.jnk]}')
 .merge(dat_constrmeta)
 .query(f'{V.COL_CONSTR_TYPE}=="Control"')
 .query(f'{V.COL_ISSIG} == 1')
)

'3 out of 44 controls have stress pathway affected'

In [ ]:
(dat_issig
 #.query(f'{V.COL_ISNB} == "oexp-NB"')
 .groupby([V.COL_CHANNELNAME, V.COL_GOODNAME,V.COL_ISNB],
          observed=True)[V.COL_ISSIG]
 .apply(lambda x: np.sum(x>0))
 .sort_values(ascending=False)
 .reset_index()
 .assign(**{V.COL_ISNB: lambda x: x[V.COL_ISNB].astype(str)})
 .set_index([V.COL_ISNB, V.COL_CHANNELNAME, V.COL_GOODNAME])[V.COL_ISSIG]
 .unstack(V.COL_ISNB)
  .sort_values('oexp-NB',ascending=False)
  .assign(**{'frac_oexp': lambda x: x['oexp-NB']/x['oexp']})
      .sort_values('frac_oexp',ascending=False)
 
 
)

In [ ]:
d= (dat_issig
 #.query(f'{V.COL_ISNB} == "oexp-NB"')
 .merge(dat_pannel_ord, left_on=V.COL_CHANNELNAME, right_on=V.COL_METAL)
 .groupby(['class',V.COL_ISNB],
          observed=True)[V.COL_ISSIG]
 .apply(lambda x: np.sum(x>0))
 .sort_values(ascending=False)
 .reset_index()
 .assign(**{V.COL_ISNB: lambda x: x[V.COL_ISNB].astype(str)})
 .set_index([V.COL_ISNB, 'class'])[V.COL_ISSIG]
 .unstack(V.COL_ISNB)
 .assign(**{'frac_oexp': lambda x: x['oexp-NB']/x['oexp']})
      .sort_values('frac_oexp',ascending=False)
)
d

In [ ]:
(dat_issig
 #.query(f'{V.COL_ISNB} == "oexp-NB"')

 .groupby([V.COL_CHANNELNAME, V.COL_GOODNAME,V.COL_ISNB],
          observed=True)[V.COL_ISSIG]
 .apply(lambda x: np.sum(x>0))
 .sort_values(ascending=False)
 .reset_index()
 .assign(**{V.COL_ISNB: lambda x: x[V.COL_ISNB].astype(str)})
 .set_index([V.COL_ISNB, V.COL_CHANNELNAME, V.COL_GOODNAME])[V.COL_ISSIG]
 .unstack(V.COL_ISNB)
  .sort_values('oexp-NB',ascending=False)
)

In [ ]:
dat_issig

How many constructs have more than one intracellular effect?

In [ ]:
oexp_thresh = 1

In [ ]:
dstat_is_respond = dstat > oexp_thresh

dstat_is_respond.head()

In [ ]:
((dstat_is_respond['other']['oexp']).to_frame()
    .join(dat_constrmeta.set_index(V.COL_CONDNAME)[[V.COL_CONSTR_TYPE]])
     .groupby(V.COL_CONSTR_TYPE)['oexp'].agg(**{'frac': lambda x: np.sum(x)/x.size,
                                        'tot': len,
                                        'pos': np.sum})
)

73% of markers affect multiple markers

How many effect > 2 in:
overexpression

In [ ]:
((dstat_is_respond['other']['oexp']).to_frame()
    .join(dat_constrmeta.set_index(V.COL_CONDNAME)[[V.COL_CONSTR_TYPE]])['oexp'].agg({'frac': lambda x: np.sum(x)/x.size,
                                        'tot': len,
                                        'pos': np.sum})
)

How many effect > 2 in:
neighbours

In [ ]:
((dstat_is_respond['other']['oexp-NB']).to_frame()
    .join(dat_constrmeta.set_index(V.COL_CONDNAME)[[V.COL_CONSTR_TYPE]])['oexp-NB'].agg({'frac': lambda x: np.sum(x)/x.size,
                                        'tot': len,
                                        'pos': np.sum})
)

How many markers are affected?

In [ ]:
dstat['other'].sort_values('oexp',ascending=False)

In [ ]:
dstat['other']['oexp-NB'].sort_values(ascending=False)

In [ ]:
((dstat_is_respond['other']['oexp-NB']).to_frame()
    .join(dat_constrmeta.set_index(V.COL_CONDNAME)[[V.COL_CONSTR_TYPE]])['oexp-NB'])

How many effect > 2 in:
bystanders

In [ ]:
((dstat_is_respond['other']['ctrl']).to_frame()
    .join(dat_constrmeta.set_index(V.COL_CONDNAME)[[V.COL_CONSTR_TYPE]])['ctrl'].agg({'frac': lambda x: np.sum(x)/x.size,
                                        'tot': len,
                                        'pos': np.sum})
)

How many intracellular markers are affected on average?

In [ ]:
(dstat['other']
 .rename(columns=str)
 .reset_index()
 .merge(dat_constrmeta)
 .groupby([V.COL_CONSTR_TYPE])['oexp'].describe()
)

How many markers are responding in the neighbourhood given significant overexpression

In [ ]:
dstat_is_respond['other'].groupby('oexp').aggregate(['mean', 'sum', len])

In [ ]:
dstat_is_respond['other'].query('(oexp == False) & (`oexp-NB` == True)')

In [ ]:
dstat_is_respond['other'].query('(oexp == False) & (`ctrl` == True)')

In [ ]:
dstat_is_respond['other'].groupby('ctrl').aggregate(['mean', 'sum', len])

(dstat_is_respond['other'].rename(columns=str)
    #.set_index('oexp')
     .join(dat_constrmeta.set_index(V.COL_CONDNAME)[[V.COL_CONSTR_TYPE]])
 .groupby([V.COL_CONSTR_TYPE,'oexp']).aggregate(['mean', 'sum','size'])
)

In [ ]:
(dstat_is_respond['other']
 .rename(columns=str)
    #.set_index('oexp')
     .join(dat_constrmeta.set_index(V.COL_CONDNAME)[[V.COL_CONSTR_TYPE]])
 .groupby(['oexp']).aggregate(['mean', 'sum','size'])
)

In [ ]:
cond_responding = dstat_is_respond['other'].query('oexp==1').rename(columns=str).reset_index()[[V.COL_CONDNAME]]

Given that a marker is significantly changed internaly, how often is it change in the neighbourhood?

In [ ]:
d=(dat_issig
    #.merge(cond_responding)
     .pivot_table(index=[V.COL_CHANNELNAME, V.COL_GOODNAME, V.COL_CONDNAME], columns=V.COL_ISNB, values=V.COL_ISSIG)
   .rename(columns=str)  
   .reset_index()
     .merge(dat_constrmeta)
       .dropna()
   .query(f'{V.COL_CHANNELNAME} not in {CHANS.stereotypic}')
     
)

In [ ]:
d.head()

In [ ]:
display(d.groupby(['oexp', V.COL_CONSTR_TYPE])['oexp-NB'].agg(**{'mn': lambda x: np.mean(x.values.astype(float)),
                                                               'sum': lambda x: np.sum(x.values.astype(float)),
                                                                 'size': len
                                                              })
       )

In [ ]:
display(d.groupby(['oexp'])['oexp-NB'].agg(**{'mn': lambda x: np.mean(x.values.astype(float)),
                                                               'sum': lambda x: np.sum(x.values.astype(float)),
                                                                 'size': len
                                                              })
       )

In [ ]:
display(d.groupby(['oexp'])['oexp-NB'].agg(**{'mn': lambda x: np.mean(x.values.astype(float)),
                                                               'sum': lambda x: np.sum(x.values.astype(float)),
                                                                 'size': len
                                                              })
       )

In [ ]:
0.131/.0146

In [ ]:
display(d.groupby(['oexp'])['ctrl'].agg(**{'mn': lambda x: np.mean(x.values.astype(float)),
                                                               'sum': lambda x: np.sum(x.values.astype(float)),
                                                                 'size': len
                                                              })
       )

In [ ]:
display(d.groupby(['oexp', V.COL_CONSTR_TYPE])['ctrl'].agg(**{'mn': lambda x: np.mean(x.values.astype(float)),
                                                               'sum': lambda x: np.sum(x.values.astype(float)),
                                                                 'size': len
                                                              })
       )

How do p38 and JNK look in the neighbourhood

In [ ]:
d=(dat_issig
    #.merge(cond_responding)
     .pivot_table(index=[V.COL_CHANNELNAME, V.COL_GOODNAME, V.COL_CONDNAME], columns=V.COL_ISNB, values=V.COL_ISSIG)
   .rename(columns=str)  
   .reset_index()
     .merge(dat_constrmeta)
       .dropna()
 #  .query(f'{V.COL_CHANNELNAME}  in {[CHANS.p38, CHANS.jnk]}')
     
)

In [ ]:
display(d.groupby(['oexp', V.COL_GOODNAME], observed=True)['oexp-NB'].agg(**{'mn': lambda x: np.mean(x.values.astype(float)),
                                                               'sum': lambda x: np.sum(x.values.astype(float)),
                                                                 'size': len
                                                              })
        .sort_values('mn', ascending=False)
        .query('size >10')
       )

In [ ]:
display(d.groupby(['oexp', V.COL_GOODNAME], observed=True)['ctrl'].agg(**{'mn': lambda x: np.mean(x.values.astype(float)),
                                                               'sum': lambda x: np.sum(x.values.astype(float)),
                                                                 'size': len
                                                              })
        .sort_values('mn', ascending=False).head(20)
        .query('size > 10')
       )

In [ ]:
display(d.groupby(['oexp', V.COL_GOODNAME], observed=True)['oexp-NB'].agg(**{'mn': lambda x: np.mean(x.values.astype(float)),
                                                               'sum': lambda x: np.sum(x.values.astype(float)),
                                                                 'size': len
                                                              })
        .sort_values('mn', ascending=False).head(20)
        .query('size > 10')
       )

Check how strong absolute changes in neighbours changed internally on average are

In [ ]:
d=(dat_absstat
 .merge(dat_issig.query(f'({V.COL_ISNB}=="oexp")&({V.COL_ISSIG}==1)')[[V.COL_CONDNAME, V.COL_CHANNELNAME]])
 .merge(cond_responding)
 .merge(dat_issig)
      .query(f'{V.COL_CHANNELNAME} not in {CHANS.stereotypic}')

)
display(d
.groupby([V.COL_ISNB, V.COL_ISSIG])['fc_vs_oexp'].describe())
display(d
.groupby([V.COL_ISNB, V.COL_ISSIG])['fc_vs_oexp']
       .apply(lambda x: np.mean(x<0)))

Check overall neighbour effects vs internal effects

In [ ]:
V.COL_MEAN_ABS_DELTA

In [ ]:
d = (dat_issig.query(f'{V.COL_ISSIG}==1')
     .merge(cond_responding)
     .assign(**{V.COL_MEAN_ABS_DELTA: lambda x: np.abs(x[V.COL_DELTA])})
     .groupby(V.COL_ISNB)[V.COL_MEAN_ABS_DELTA].describe()
)
display(d)
display(10**d)

How many of these are actually the overexpressed gene?

-> merch gene pannel and gene constructmeta to get all combinations

In [ ]:
V.COL_GENE = 'gene'
V.COL_METAL = 'metal'

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
dat_pannel.columns

In [ ]:
d=(dat_issig.merge(dat_constrmeta[[V.COL_CONDNAME, V.COL_GENE]])
     .merge(dat_pannel, left_on=V.COL_CHANNELNAME, right_on=V.COL_METAL)
     .query(f'{V.COL_GENE}_x== {V.COL_GENE}_y')
     .query(f'{V.COL_GENE}_x !="GFP"')
)   

display(d.query(f'{V.COL_ISNB} == "oexp"')[[V.COL_ISSIG,  V.COL_CONDNAME, V.COL_MODIFICATION,
                                            V.COL_ISNB, V.COL_PLTNAME]])
d.groupby([V.COL_ISNB,'modification'])[V.COL_ISSIG].agg(**{'mn': 'mean',
                                                            'size': 'size'})

Of all things without a neighbourhood effect but with a bystander effect: what direction does the effect go?

In [ ]:
(dat_issig
     .pivot_table(index=[V.COL_CHANNELNAME, V.COL_GOODNAME, V.COL_CONDNAME], columns=V.COL_ISNB, values='sigdir')
        .rename(columns=str)  
     .reset_index()
)

In [ ]:
d = (dat_issig
     .assign(**{'sigdir': lambda x: x['sigdir'].astype(str).astype(float)})
     .pivot_table(index=[V.COL_CHANNELNAME, V.COL_GOODNAME, V.COL_CONDNAME], columns=V.COL_ISNB, values='sigdir')
        .rename(columns=str)  
     .reset_index()
     .merge(dat_constrmeta)
       .dropna()
)
d['any'] = (d['oexp-NB'] != 0) | (d['oexp'] != 0)

display(d.groupby(['any', 'ctrl']).describe())

In [ ]:
d.loc[(d['oexp-NB'] == 0) & (d['ctrl'] == 1),:]

How many respond in the neibourhood?

In [ ]:
((dstat['other']['oexp'] >1).to_frame()
    .join(dat_constrmeta.set_index(V.COL_CONDNAME)[[V.COL_CONSTR_TYPE]])
     .groupby(V.COL_CONSTR_TYPE)['oexp'].agg(**{'frac': lambda x: np.sum(x)/x.size,
                                        'tot': 'size',
                                        'pos': 'sum'})
)

In [ ]:
dstat.stack(level=V.COL_ISSTEREOTYPIC).groupby(V.COL_ISSTEREOTYPIC)['oexp'].apply(lambda x: np.mean(x>0))

Which overexpression has the largest effect on the environment?

Check model convergence

In [ ]:
dat_model= (pd.concat([pd.read_csv(fn) for fn in C.fns_lmm_model],
                        keys=chans, names=[V.COL_CHANNELNAME,'index']).reset_index(V.COL_CHANNELNAME, drop=False)
               .reset_index(drop=True)
                )

In [ ]:
dat_model

-> All converged

In [ ]:
dat_modmeta= (pd.concat([pd.read_csv(fn) for fn in C.fns_lmm_model],
                        keys=chans, names=[V.COL_CHANNELNAME,'index']).reset_index(V.COL_CHANNELNAME, drop=False)
               .reset_index(drop=True)
                )

In [ ]:
dat_randeff = (pd.concat([pd.read_csv(fn) for fn in C.fns_lmm_randeff],
                        keys=chans, names=[V.COL_CHANNELNAME,'index']).reset_index(V.COL_CHANNELNAME, drop=False)
               .reset_index(drop=True)
                )

In [ ]:
dat_randeff.head()

In [ ]:
(dat_randeff
 
     #.query('grp not in ["Residual"]')
     #.query('grp=="site_id"')
      .merge(dat_pannel[[V.COL_METAL, V.COL_GOODNAME]], left_on=V.COL_CHANNELNAME, right_on=V.COL_METAL)
     .assign(fc_sdcor=lambda x: 10**x['sdcor'])
     .assign(idx=lambda x: 2.5+np.linspace(2.5,10,int(x.shape[0]))) >>
     gg.ggplot(gg.aes(x='fc_sdcor')) +
     gg.facet_wrap('grp')+
     gg.geom_histogram(bins=30)+
     gg.scale_x_log10(breaks=(1,1.05,1.1,1.2,1.3,1.4,1.5,1.6,1.7,2, 2.5, 3, 4, 6, 8, 10))
     + gg.geom_text(gg.aes(y=8, label=V.COL_GOODNAME),angle=90, size=6)
     + gg.ggtitle('Variance contributions')
     + gg.xlab('Standard deviation [Fold Change]')
     + gg.theme(figure_size=(15,8))

)